In [1]:
# Community simulator package
from IPython.display import Image
from community_simulator import *
from community_simulator.usertools import *
from community_simulator.visualization import *
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends import backend_pdf as bpdf
import numpy as np
import scipy as sp
colors = sns.color_palette()
%matplotlib inline

# Community selection package
from community_selection import *
from community_selection.A_experiment_functions import *
from community_selection.B_community_phenotypes import *
from community_selection.C_selection_algorithms import *
from community_selection.D_migration_algorithms import *

In [2]:
#make_algorithm_library()

# Simulation setting
Below is parameters that determine the universe for simulation

In [3]:
# Make dynanmics by default we will use the microbial consumer resource model
def dNdt(N,R,params):
    return MakeConsumerDynamics(assumptions)(N,R,params)
def dRdt(N,R,params):
    return MakeResourceDynamics(assumptions)(N,R,params)
dynamics = [dNdt,dRdt]

# Global parameters
## Default parameters from community-simulator
## !!!Don't touch this dictionary!!!
assumptions = a_default.copy() # Start with default parameters
a_default = {'sampling':'Binary', #{'Gaussian','Binary','Gamma'} specifies choice of sampling algorithm
            'SA': 60*np.ones(3), #Number of species in each specialist family (here, 3 families of 60 species)
            'MA': 30*np.ones(3), #Number of resources in each class 
            'Sgen': 30, #Number of generalist species (unbiased sampling over alll resource classes)
            'muc': 10, #Mean sum of consumption rates (used in all models)
            'sigc': 3, #Standard deviation of sum of consumption rates for Gaussian and Gamma models
            'q': 0.0, #Preference strength of specialist families (0 for generalist and 1 for specialist)
            'c0':0.0, #Sum of background consumption rates in binary model
            'c1':1., #Specific consumption rate in binary model
            'l':0.8, #Leakage fraction
            'fs':0.45, #Fraction of secretion flux with same resource type
            'fw':0.45, #Fraction of secretion flux to 'waste' resource
            'sparsity':0.2, #Effective sparsity of metabolic matrix (between 0 and 1)
            'n_wells':10, #Number of independent wells
            'S':100, #Number of species per well (randomly sampled from the pool of size Stot = sum(SA) + Sgen)
            'food':0, #index of food source (when a single resource is supplied externally)
            'R0_food':1000, #unperturbed fixed point for supplied food
            'regulation':'independent', #metabolic regulation (see dRdt)
            'response':'type I', #functional response (see dRdt)
            'supply':'off' #resource supply (see dRdt)
            }

## Update parameters for community-selection
assumptions.update({
    'SA': 600*np.ones(3), #Number of species in each specialist family (here, 3 families of 60 species)
    'MA': 300*np.ones(3), #Number of resources in each class 
    'Sgen': 300, #Number of generalist species (unbiased sampling over alll resource classes)
    "n_wells": 24,
    "c1": 1, #Rescale uptake rate part 1. This is needed to avoid numerical errors that slow down the simulations
    "muc": 10, # Rescale uptake part 2
    "m": 0, # Mortality
    "scale": 10**9,  #scale is a conversion factor specifying the number of individual microbial cells present when N = 1.
    "sigma" : 1, # Standard deviation for drawing specifc speices/interaction function
    "alpha": 1, # Scaling factor between species- and interaction-specific function variances
    "response": "type I",
    "sigma_max": 1, 
    'R0_food':1000 #unperturbed fixed point for supplied food
})



# Prepare experiment setup in this universe
params, species_pool, species_function, interaction_function = prepare_experiment(assumptions, seed = 1)

## Simulation parameters
params_simulation = {
    "n_propagation": 8, # Length of propagation, or hours within a growth cycle
    "n_transfer": 20, # Number of transfer, or number of passage
    "dilution": 1/125, # Dilution factor at every transfer
    "n_inoc": 128,  #Number of cells sampled from the regional species at start
    "pool": species_pool, 
    "species_function": species_function,
    "interaction_function": interaction_function,
    "selected_function": "f1_additive"
}

## Community selection

Test each of the community function under different algorithms

- After selection, communities are allowed to stabilized for addtition 10 transfers 

In [4]:
data_directory = "data/typeI-24wells-large_pool/"
list_phenotypes = ["f1_additive", "f2_interaction", "f3_additive_binary", "f4_interaction_binary"]
list_algorithms = ["simple_screening", "swenson2000", "direct_selection"]
#list_algorithms = ["simple_screening", "direct_selection", "select_top25", "select_top10", "multiple_direct_selection", "pair_top_communities", "swenson2000"]


In [5]:
for j in range(len(list_phenotypes)):
    # Parameters for simulation
    params_simulation.update({"selected_function": list_phenotypes[j]}) # selected function

    # Make the list of algorithms
    algorithms = make_algorithms(params_simulation)
    
    # Simulation
    for i in range(len(list_algorithms)):
        simulate_community(
            assumptions = assumptions,
            params = params,
            dynamics = dynamics,
            params_simulation = params_simulation, 
            params_algorithm = algorithms[algorithms["algorithm_name"] == list_algorithms[i]], 
            write_composition = True,
            file_name = data_directory + list_algorithms[i],
            assembly_type = str(list_algorithms[i]),
        )
        


Algorithm: simple_screening


 transfer community_phenotype selection_algorithm migration_algorithm
        1         f1_additive        no_selection        no_migration
        2         f1_additive        no_selection        no_migration
        3         f1_additive        no_selection        no_migration
        4         f1_additive        no_selection        no_migration
        5         f1_additive        no_selection        no_migration
        6         f1_additive        no_selection        no_migration
        7         f1_additive        no_selection        no_migration
        8         f1_additive        no_selection        no_migration
        9         f1_additive        no_selection        no_migration
       10         f1_additive        no_selection        no_migration
       11         f1_additive        no_selection        no_migration
       12         f1_additive        no_selection        no_migration
       13         f1_additive        no_selection        no

/Users/chang-yu/anaconda3/lib/python3.7/site-packages/community_simulator/usertools.py:392: RuntimeWarning: invalid value encountered in multiply
  *params['w']*sigma[assumptions['response']](R,params))
/Users/chang-yu/anaconda3/lib/python3.7/site-packages/community_simulator/usertools.py:380: RuntimeWarning: invalid value encountered in multiply
  sigma = {'type I': lambda R,params: params['c']*R,
/Users/chang-yu/anaconda3/lib/python3.7/site-packages/community_simulator/usertools.py:356: RuntimeWarning: invalid value encountered in multiply
  *params['w']*sigma[assumptions['response']](R,params))
/Users/chang-yu/anaconda3/lib/python3.7/site-packages/community_simulator/usertools.py:338: RuntimeWarning: invalid value encountered in multiply
  sigma = {'type I': lambda R,params: params['c']*R,
Process ForkPoolWorker-7:
Process ForkPoolWorker-5:
Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
Process ForkPoolWorker-3:
Traceback (most recent call last):
  Fil

  File "<ipython-input-3-65fdb16362f2>", line 5, in dRdt
    return MakeResourceDynamics(assumptions)(N,R,params)
  File "/Users/chang-yu/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py", line 244, in odeint
    int(bool(tfirst)))
  File "/Users/chang-yu/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py", line 244, in odeint
    int(bool(tfirst)))
  File "/Users/chang-yu/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py", line 244, in odeint
    int(bool(tfirst)))
  File "/Users/chang-yu/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py", line 244, in odeint
    int(bool(tfirst)))
  File "/Users/chang-yu/anaconda3/lib/python3.7/site-packages/community_simulator/usertools.py", line 357, in <lambda>
    J_out = lambda R,params: (params['l']*J_in(R,params)).dot(params['D'].T)
  File "/Users/chang-yu/anaconda3/lib/python3.7/site-packages/community_simulator/usertools.py", line 361, in <lambda>
    +(J_out(R,params)/params['w']).

KeyboardInterrupt: 